## Langchain의 OpenAI GPT모델 사용법

#### LangChain: 대규모 언어 모델들을 중심으로 구축된 프레임워크
- 사용자의 질문과 LLM의 답변 사이에 여러 가지 Tool을 이용해 프롬프트 엔지니어링을 하는 것. 그리고 이러한 과정을 반복하여 원하는 답을 찾는 것
1. 프롬프트 템플릿: 챗봇 스타일 템플릿, ELI5 질문-답변 등 다양한 유형의 프롬프트를 위한 템플릿
2. LLMs: GPT-3, BLOOM 등과 같은 큰 언어 모델이다.
3. 에이전트: LLMs를 사용하여 어떤 행동이 취해져야 하는지 결정하는 도구(LLM을 사용하여 프롬프트를 행동 계획으로 추출하는 React 프롬프트 제작을 단순화 함)
      - LangChain에서 강력한 모듈
      - React Prompt 기법: LLM의 추론과 행동
- LangChain 모듈
 - memory: chains/agents 호출 사이에 application의 상태를 유지
 - callbacks: chain의 중간 단계를 기록하거나 스트리밍 함
 - Data connection: application 특화된 데이터와 인터페이스
 - Model I/O: 언어모델과 인터페이스
 - Agent: chain이 high-level의 지침에 따라 사용할 도구를 선택하도록 함
 - Chains: 호출 시퀀스를 구성

#### Langchain의 주요 기능
1. 문맥 인식: 언어 모델과 다양한 문맥소스를 연동하며, 사용자의 문맥을 정확히 이해한다.
2. 추론 능력: 제공된 문맥에 기반하여 어떤 대답을 할지, 또는 어떤 액션을 취할지에 대한 추론이 가능하다.

#### Langchain의 핵심 가치 중 두가지 주요한 점
1. 구성 요소: 사용자는 언어 모델과의 상호작용을 위해 다양한 구성 요소와 추상화를 활용할 수 있다. 구성 요소는 개별적으로, 또는 랭체인 프레임워크 내에서 모듈식으로 쉽게 활용 가능
2. 사용 준비된 체인: 특정 고수준 작업을 수행하기 위해 미리 조립된 구성 요소의 패키지

openai의 chatgpt api 발급받기
- api 비용 지불 문제로 인해 아래의 코드 결과 확인 어려움...


In [6]:
!pip install openai langchain

In [7]:
!pip install openai==0.28

In [8]:
import os

os.environ["OPENAI_API_KEY"]="발급한 api"

In [9]:
# 사용 가능한 모델 리스트 출력
import openai

model_list = sorted([m['id'] for m in openai.Model.list()['data']])
for m in model_list:
  print(m)

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


### ChatOpenAI
#### Hyperparameters
- temperature: 사용할 샘플링 온도 0~2 값 선택. 0.8과 같은 높은 값은 출력을 더 무작위 하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만든다.
- max_tokens: 채팅 완성에서 생성할 토큰의 최대 개수
- model_name: 적용 가능한 모델 리스트

In [10]:
!pip install langchain

In [11]:
from langchain.chat_models import ChatOpenAI

# 객체 생성
llm= ChatOpenAI(temperature=0, # 창의성 (0.0 ~ 2.0)
                max_tokens=2048, # 최대 토큰수
                model_name="gpt-3.5-turbo", # 모델명
                )
# 질의내용
question="대한민국의 수도는 뭐야?"
# 질의
print(f'[답변]: {llm.predict(question)}')


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

### 프롬프트 템플릿의 활용
PromptTemplate
- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는데 사용되는 템플릿
- 사용법
 - template: 템플릿 문자열, 중괄호 {}는 변수를 나타낸다.
 - input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의

input_variables
- PromptTemplate에 사용되는 변수의 이름을 정의하는 리스트
- 리스트 형식으로 변수 이름을 정의

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 질문 템플릿 형식 정의
template = '{country}의 수도는 뭐야?'

# 템플릿 완성
prompt=PromptTemplate(template=template, input_variables=['country'])

### LLMChain 객체
- LLMChain은 특정 PromptTemplate과 연결된 체인 객체를 생성한다.
- 사용법
  - prompt: 앞서 정의한 PromptTemplate 객체를 사용
  - llm: 언어 모델을 정의
  

In [13]:
# 연결된 체인(chain)객체 생성
llm_chain=LLMChain(prompt=prompt, llm=llm)

In [14]:
# 체인 실행: run()
print(llm_chain.run(country="일본"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
# 체인 실행: run()
print(llm_chain.run(country='캐나다'))

In [18]:
# apply() 함수를 사용하여 여러개의 입력을 한 번에 실행 가능
input_list=[{'country':'영국','country':'프랑스','country':'미국'}]
llm_chain.apply(input_list)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [19]:
# input_list에 대한 결과 반환
result=llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
  print(res['text'].strip())

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [23]:
# generate: 문자열 대신에 LLMResult를 반환하는 점을 제외하고 apply와 유사하다.
# input_list에 대한 결과 반환
generated_result=llm_chain.generate(input_list)
print(generate_result)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [27]:
# 답변 출력
generated_result.generations

NameError: name 'generated_result' is not defined

In [24]:
# 토큰 사용량 출력
generated_result.llm_output

NameError: name 'generated_result' is not defined

In [25]:
# run ID 출력
generated_result.run

NameError: name 'generated_result' is not defined

In [26]:
# 답변 출력
for gen in generated_result.generations:
  print(gen[0].text.strip())

NameError: name 'generated_result' is not defined

## Langchain + HuggingFace 모델 사용법


HuggingFace에서 발급받은 LLM key:

In [29]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"]='huggingface llm read key'

In [36]:
# HuggingFaceHub에 배포된 모델 추론(inference)
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub

# HuggingFace Repository ID
repo_id='mistralai/Mistral-7B-v0.1'

# 질의내용
question="Who is Son Heung Min?"

# 템플릿
template="""Question: {question}
Answer: """

# 프롬프트 템플릿 생성
prompt=PromptTemplate(template=template, input_variables=["question"])

In [39]:
# HuggingFaceHub 객체 생성
llm=HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature":0.2,
                  "max_length":128}
)

# LLM Chain 객체 생성
llm_chain=LLMChain(prompt=prompt,llm=llm)

# 실행
print(llm_chain.run(question=question))

Question: Who is Son Heung Min?
Answer: 2018 World Cup Golden Boot winner.

Question: Who is Son Heung Min?
Answer: 2018 World Cup Golden Boot winner.

Question: Who is Son Heung Min?
Answer: 2018 World Cup Golden Boot winner.

Question: Who is Son Heung Min?
Answer: 2018 World Cup Golden Boot winner.

Question: Who is Son Heung Min?



In [47]:
# HuggingFace Repository ID
repo_id='google/flan-t5-xxl'

# 질의내용
question="who is Son Heung Min?"

# 템플릿
template=""" Question: {question}
Answer:
"""

# 프롬프트 템플릿 생성
prompt=PromptTemplate(template=template,input_variables=["question"])

# HuggingFaceHub 객체 생성
llm=HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature":0.2,
                 "max_length":512}
)

# LLM Chain 객체 생성
llm_chain=LLMChain(prompt=prompt, llm=llm)

# 실행
print(llm_chain.run(question=question))




South Korean footballer
